In [1]:
FOLDER_ROOT = 'F:\Github\ml-phd-research'
import sys
sys.path.append(FOLDER_ROOT)

In [4]:
pip install numba

In [2]:
import numpy as np

import pandas as pd

import timeit

from sklearn.metrics import accuracy_score

In [5]:
from midas.v4.ensemble import RFClassifier

NameError: name 'njit' is not defined

# Testes

Verifica se instância com os valores padrão

In [4]:
rf = RFClassifier()

In [5]:
print(dir(rf))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_adapt_massive_inputs', '_base_estimator', '_build_estimator', '_calc_boostrap_size', '_check_feature_names', '_check_inputs', '_check_n_features', '_classes', '_estimators', '_feature_estimators', '_feature_names', '_generate_sample_indices', '_get_param_names', '_get_sample_data', '_get_sample_n_feature', '_get_tags', '_make_estimator', '_more_tags', '_n_classes', '_n_estimators', '_n_features', '_n_jobs', '_params', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_sample_bootstrap_size', '_sample_n_feature', '_sample_strategy', '_seed', '_validate_data', '_verbose', 'fit', 'get_params', 'pr

In [6]:
rf._base_estimator

sklearn.tree._classes.DecisionTreeClassifier

In [7]:
rf._n_estimators

100

In [8]:
rf._params

{'criterion': 'entropy',
 'splitter': 'best',
 'max_depth': None,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'min_weight_fraction_leaf': 0.0,
 'max_features': 'auto',
 'random_state': 100,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'class_weight': None,
 'ccp_alpha': 0.0}

In [9]:
rf._estimators

[]

Teste do fit

In [10]:
df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

In [11]:
X.head()

,salary,commission,age,elevel,car,zipcode,hvalue,hyears,loan
0,110499.735409,0.000000,54,3,6,4,135000.0,30,354724.182530
1,140893.779095,0.000000,44,4,12,7,135000.0,2,395015.339020
2,119159.651677,0.000000,49,2,0,3,135000.0,22,122025.085242
3,20000.000000,52593.636537,56,0,19,1,135000.0,30,99629.621457
4,93956.320488,0.000000,59,1,6,1,135000.0,1,458679.826238


In [12]:
rf = rf.fit(X, y)

In [14]:
rf._feature_estimators

array([list([3, 4, 1]), list([4, 0, 3]), list([1, 3, 2]), list([1, 4, 3]),
       list([5, 1, 3]), list([7, 4, 8]), list([7, 5, 8]), list([7, 3, 0]),
       list([8, 2, 6]), list([8, 7, 6]), list([0, 6, 4]), list([5, 3, 8]),
       list([6, 2, 3]), list([8, 5, 3]), list([7, 4, 3]), list([2, 8, 1]),
       list([1, 2, 8]), list([0, 8, 3]), list([4, 3, 8]), list([6, 8, 5]),
       list([7, 4, 8]), list([4, 3, 1]), list([1, 7, 2]), list([1, 6, 0]),
       list([2, 1, 7]), list([6, 3, 1]), list([2, 3, 8]), list([5, 6, 4]),
       list([7, 4, 3]), list([3, 1, 6]), list([1, 7, 6]), list([1, 2, 7]),
       list([2, 8, 6]), list([4, 0, 5]), list([6, 4, 2]), list([2, 6, 0]),
       list([7, 5, 4]), list([8, 1, 0]), list([4, 2, 8]), list([3, 2, 8]),
       list([7, 2, 4]), list([8, 1, 5]), list([8, 1, 5]), list([6, 2, 5]),
       list([7, 8, 6]), list([3, 5, 4]), list([5, 8, 3]), list([1, 7, 4]),
       list([4, 1, 3]), list([7, 6, 8]), list([0, 2, 1]), list([4, 3, 2]),
       list([7, 0, 8]), l

Verifica o tempo de execução para diferentes configurações de paralelismo no método fit.

In [4]:
df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

In [5]:
# parallel_backend='loky', parallel_prefer=None, parallel_require=None
rf = RFClassifier()rf = RFClassifier()

In [6]:
%timeit rf.fit(X, y)

AttributeError: _base_estimator

In [8]:
# parallel_backend='multiprocessing', parallel_prefer=None, parallel_require=None
rf = RFClassifier(parallel_backend='multiprocessing', parallel_prefer=None, parallel_require=None)

In [9]:
%timeit rf.fit(X, y)

AttributeError: _base_estimator

In [10]:
# parallel_backend='multiprocessing', parallel_prefer=None, parallel_require='sharedmem'
rf = RFClassifier(parallel_backend='multiprocessing', parallel_prefer=None, parallel_require='sharedmem')

In [11]:
%timeit rf.fit(X, y)

ValueError: Backend <joblib._parallel_backends.MultiprocessingBackend object at 0x000000BF3D4B6BA8> does not support shared memory

In [12]:
# parallel_backend='threading', parallel_prefer=None, parallel_require=None
rf = RFClassifier(parallel_backend='threading', parallel_prefer=None, parallel_require=None)

In [13]:
%timeit rf.fit(X, y)

4min 32s ± 25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# parallel_backend='threading', parallel_prefer='processes', parallel_require=None
rf = RFClassifier(parallel_backend='threading', parallel_prefer='processes', parallel_require=None)

In [15]:
%timeit rf.fit(X, y)

4min 37s ± 23.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# parallel_backend='threading', parallel_prefer='processes', parallel_require='sharedmem'
rf = RFClassifier(parallel_backend='threading', parallel_prefer='processes', parallel_require='sharedmem')

In [17]:
%timeit rf.fit(X, y)

ValueError: prefer == 'processes' and require == 'sharedmem' are inconsistent settings

In [18]:
# parallel_backend='threading', parallel_prefer=None, parallel_require='sharedmem'
rf = RFClassifier(parallel_backend='threading', parallel_prefer=None, parallel_require='sharedmem')

In [19]:
%timeit rf.fit(X, y)

4min 24s ± 29.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
# parallel_backend='threading', parallel_prefer='threads', parallel_require=None
rf = RFClassifier(parallel_backend='threading', parallel_prefer='threads', parallel_require=None)

In [21]:
%timeit rf.fit(X, y)

4min 1s ± 18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
# parallel_backend='threading', parallel_prefer='threads', parallel_require='sharedmem'
rf = RFClassifier(parallel_backend='threading', parallel_prefer='threads', parallel_require='sharedmem')

In [23]:
%timeit rf.fit(X, y)

4min 6s ± 16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
rf._feature_estimators[0]

[5, 4, 1]

In [28]:
rf._feature_names is None

True

Testa o predict

In [4]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [5]:
rf = RFClassifier()

In [6]:
rf = rf.fit(X_train, y_train)

Faz um teste simples de chamada do predict.

In [13]:
y_pred = rf.predict(X_test)

In [8]:
len(y_pred)

700000

In [9]:
y_pred[:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)

In [10]:
# 1 execução
accuracy_score(y_test, y_pred)

0.6332757142857143

In [8]:
# 2 execução
accuracy_score(y_test, y_pred)

0.6383671428571429

In [10]:
# 3 execução
accuracy_score(y_test, y_pred)

0.6383671428571429

In [12]:
# 4 execução
accuracy_score(y_test, y_pred)

0.6383671428571429

In [14]:
# 5 execução
accuracy_score(y_test, y_pred)

0.6383671428571429

Verifica o tempo de execução para diferentes configurações de paralelismo no método predict.

In [4]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [5]:
rf = RFClassifier()

In [6]:
rf = rf.fit(X_train, y_train)

In [17]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer=None, parallel_require=None)
%timeit rf.predict(X_test)

31.9 s ± 3.74 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer='processes', parallel_require=None)
%timeit rf.predict(X_test)

35.7 s ± 3.77 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer='processes', parallel_require='sharedmem')
%timeit rf.predict(X_test)

ValueError: prefer == 'processes' and require == 'sharedmem' are inconsistent settings

In [11]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer=None, parallel_require='sharedmem')
%timeit rf.predict(X_test)

39.6 s ± 3.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer='threads', parallel_require=None)
%timeit rf.predict(X_test)

29.2 s ± 1.61 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer='threads', parallel_require='sharedmem')
%timeit rf.predict(X_test)

31 s ± 5.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Verifica se as mudanças dos parâmetros de paralelismo afetam o acurácia.

In [19]:
rf = RFClassifier()
rf = rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6299328571428572


In [20]:
rf.set_parallel_options(parallel_backend='threading', parallel_prefer=None, parallel_require=None)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6299328571428572


Executa o teste após mudar o contexto para o build da árvore.

Referência:

        fit -> 4min 1s ± 18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    predict -> 4min 1s ± 18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
        acc -> 0.6383671428571429 ou 0.6332757142857143

In [4]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [5]:
rf = RFClassifier()

80% paralelizável do código

In [6]:
%timeit rf.fit(X_train, y_train)

1min 3s ± 3.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit rf.predict(X_test)

24.4 s ± 3.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
rf = rf.fit(X_train, y_train)

In [9]:
y_pred = rf.predict(X_test)

In [10]:
accuracy_score(y_test, y_pred)

0.6391742857142857

100% paralelizável do código

In [6]:
%timeit rf.fit(X_train, y_train)

1min 5s ± 5.23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit rf.predict(X_test)

22 s ± 1.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


90% paralelizável do código

In [6]:
%timeit rf.fit(X_train, y_train)

1min 1s ± 5.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit rf.predict(X_test)

24.9 s ± 2.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
accuracy_score(y_test, y_pred)

Executa o teste após implementar o gc manualmente.

Referência:

    fit -> 1min 3s ± 3.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    predict -> 24.4 s ± 3.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    acc -> 0.6391742857142857

In [4]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [5]:
rf = RFClassifier()

In [6]:
%timeit rf.fit(X_train, y_train)

1min 26s ± 5.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
rf = rf.fit(X_train, y_train)

In [8]:
%timeit rf.predict(X_test)

29.2 s ± 1.86 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
y_pred = rf.predict(X_test)

In [10]:
accuracy_score(y_test, y_pred)

0.6479071428571429

In [14]:
# importing the module
import tracemalloc

# starting the monitoring
tracemalloc.start()

# function call
rf = rf.fit(X_train, y_train)

# displaying the memory
print(tracemalloc.get_traced_memory())
 
# stopping the library
tracemalloc.stop()

(95521, 206536108)


In [15]:
pip install -U memory_profiler

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: memory_profiler in c:\users\danilo santos\appdata\local\programs\python\python37\lib\site-packages (0.60.0)
  Attempting uninstall: memory_profiler
    Found existing installation: memory-profiler 0.60.0
    Uninstalling memory-profiler-0.60.0:
      Successfully uninstalled memory-profiler-0.60.0




[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from memory_profiler import profile

In [17]:
@profile
rf = RFClassifier()

rf = rf.fit(X_train, y_train)

SyntaxError: invalid syntax (46634503.py, line 2)

In [4]:
import time
import resource 

rf = RFClassifier()

time_start = time.perf_counter()

rf = rf.fit(X_train, y_train)

time_elapsed = (time.perf_counter() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print ("%5.1f secs %5.1f MByte" % (time_elapsed,memMb))

# sample output:
# 2.3 secs 140.8 MByte

ModuleNotFoundError: No module named 'resource'

In [5]:
%load_ext memory_profiler

%memit rf = RFClassifier()

peak memory: 398.76 MiB, increment: 0.18 MiB


In [8]:
%memit 
rf = rf.fit(X_train, y_train)

peak memory: 429.53 MiB, increment: 0.00 MiB


In [6]:
%memit rf = rf.fit(X_train, y_train)

peak memory: 1099.09 MiB, increment: 698.28 MiB


Executa o teste após implementar o logging manualmente.

Referência:

    fit -> 1min 3s ± 3.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    predict -> 24.4 s ± 3.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    acc -> 0.6391742857142857

In [4]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [5]:
rf = RFClassifier(enable_logger=True)

In [6]:
%timeit rf.fit(X_train, y_train)

1min 11s ± 5.16 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit rf.predict(X_test)

32.4 s ± 4.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
rf = rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

# Teste atual

Executa o teste usando numba.

Referência:

    fit -> 1min 3s ± 3.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    predict -> 24.4 s ± 3.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    acc -> 0.6391742857142857

In [ ]:
from sklearn.model_selection import train_test_split

df_stream = pd.read_csv('https://github.com/scikit-multiflow/streaming-datasets/raw/master/agr_a.csv',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=100)

In [ ]:
rf = RFClassifier()

In [ ]:
rf = rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [6]:
from numba import jit

@jit
def foo(a):
    return a*a 

In [7]:
def goo(a):
    return sum([for i in range(a)])

@jit(nopython=True, nogil=True, cache=True, parallel=True)=goo

SyntaxError: invalid syntax (1788022235.py, line 2)